Черняев Александр Павлович

## Описание проекта
Сервис  “Мой Чемпион” помогает спортивным школам фигурного катания, тренерам мониторить результаты своих подопечных и планировать дальнейшее развитие спортсменов.
### Цель
Создать решение для стандартизации названий спортивных школ.

### Исходные данные
- Эталонные написания названий
- Приблизительные написания названий

## Решение

Импортируем библиотеки и модули

In [1]:
import pandas as pd
import numpy as np

import re
import random
import requests

from sklearn.model_selection import train_test_split

from sentence_transformers import SentenceTransformer, InputExample, util, losses

C:\Anaconda\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
# Загрузим датасет с примерными написаниями названий
approximate_names = pd.read_csv('approximate.csv')

In [3]:
approximate_names.head()

,school_id,name
0,1836,"ООО ""Триумф"""
1,1836,"Москва, СК ""Триумф"""
2,610,"СШОР ""Надежда Губернии"
3,610,"Саратовская область, ГБУСО ""СШОР ""Надежда Губе..."
4,609,"""СШ ""Гвоздика"""


In [4]:
# Загрузим датасет с эталонными написаниями названий
school_names = pd.read_csv('schools.csv')

In [5]:
school_names.head()

,school_id,name,region
0,1,Авангард,Московская область
1,2,Авангард,Ямало-Ненецкий АО
2,3,Авиатор,Республика Татарстан
3,4,Аврора,Санкт-Петербург
4,5,Ice Dream / Айс Дрим,Санкт-Петербург


In [6]:
school_names.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306 entries, 0 to 305
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   school_id  306 non-null    int64 
 1   name       306 non-null    object
 2   region     306 non-null    object
dtypes: int64(1), object(2)
memory usage: 7.3+ KB


In [7]:
# Создадим рабочий датасет из датасета с эталонными написаниями названий
df = school_names.copy()

Создадим образец написания названия школы по типу "Регион, Название школы" в колонке `title`

In [8]:
df['title'] = df['region'] + ' ' + df['name']

Оставим только колонки `school_id` и `title`

In [9]:
df = df[['school_id', 'title']]

Напишем функцию для аугментации написаний названий

In [10]:
# Функция для аугментации
def aug(word):
    # Пропущена каждая 3-я буква
    a1 = ''.join([x for i,x in enumerate(word) if i%3 != 2])

    # Удалены все гласные буквы
    a2 = ''.join(i for i in word if i not in 'АаЕеЁёИиОоУуЫыЭэЮюЯя')
    a2 = ' '.join(a2.split())

    #Удаление случайного элемента из каждого слова
    a3 = ''
    l = word.split(' ')
    for w in l:
        if len(w) < 2:
            a3 = w
        else:
            i = random.randint(0, len(w)-2)
            new_word = w.replace(w[i], '', 1)
            a3 += new_word + ' '
    a3 = a3[:-1]
            #a3 = ' '.join(a3.split())

    # Случайные соседние буквы поменяны местами в каждом слове
    a4 = ''
    l = word.split(' ')
    for w in l:
        if len(w) < 2:
            a4 = w
        else:
            i = random.randint(0, len(w)-2)
            new_word = w[:i] + w[i + 1] + w[i] + w[i + 2:]
            a4 += new_word + ' '
    a4 = a4[:-1]
    
    return [a1, a2, a3, a4]

In [11]:
# Применяем функцию
df['augmented'] = df['title'].transform(lambda x: aug(x))

In [12]:
# Смотрим на результат
df.head(20)

,school_id,title,augmented
0,1,Московская область Авангард,"[Мокоскя блст Аанар, Мсквск блсть внгрд, Моско..."
1,2,Ямало-Ненецкий АО Авангард,"[ЯмлоНеецийАОАвнгрд, мл-Ннцкй внгрд, ЯмалоНене..."
2,3,Республика Татарстан Авиатор,"[Репулиа атрсанАватр, Рспблк Ттрстн втр, Рспуб..."
3,4,Санкт-Петербург Аврора,"[СактПеерур Ароа, Снкт-Птрбрг врр, анкт-Петерб..."
4,5,Санкт-Петербург Ice Dream / Айс Дрим,"[СактПеерур Ie rem Ас ри, Снкт-Птрбрг Ice Dre..."
5,6,Республика Крым Айсберг,"[Репулиа ры Асбрг, Рспблк Крм йсбрг, Республиа..."
6,7,Рязанская область Айсберг,"[Ряанка олатьАйбег, Рзнск блсть йсбрг, Рзанска..."
7,8,Свердловская область Айсберг,"[Сврдовка олатьАйбег, Сврдлвск блсть йсбрг, Се..."
8,9,Северодвинск Звездочка,"[Сеердвнс ведока, Сврдвнск Звздчк, Звздочка, ..."
9,10,Москва Академия синхронного катания на коньках,"[Мокв Аадми снхоног ктаиянакоькх, Мскв кдм снх..."


In [13]:
# Добавим в df['augmented'] название школы без региона
for i in range(df.shape[0]):
    df['augmented'][i].append(school_names['name'][i])

In [14]:
df_augmented = df.explode('augmented')[['title', 'augmented']].reset_index(drop=True)

In [16]:
df_augmented

,title,augmented
0,Московская область Авангард,Мокоскя блст Аанар
1,Московская область Авангард,Мсквск блсть внгрд
2,Московская область Авангард,Москоская обасть вангард
3,Московская область Авангард,Московксая облсать Аванагрд
4,Московская область Авангард,Авангард
...,...,...
1525,"Москва ООО ""Триумф""","Мокв ОО Трум"""
1526,"Москва ООО ""Триумф""","Мскв ""Трмф"""
1527,"Москва ООО ""Триумф""","Мосва ОО ""Тиумф"""
1528,"Москва ООО ""Триумф""","Мосвка ООО ""Триуфм"""


Создадим тренировочную и тестовую выборки

In [17]:
train, test = train_test_split(df_augmented, test_size = 0.2, random_state = 12345)

Выведем размеры выборок

In [18]:
train.shape

(1224, 2)

In [19]:
test.shape

(306, 2)

Объявляем модель

In [20]:
model = SentenceTransformer('sentence-transformers/LaBSE')

In [21]:
# Оставим в датасете df только колонки 'title' и 'augmented'
df = df[['title', 'augmented']]

In [22]:
df

,title,augmented
0,Московская область Авангард,"[Мокоскя блст Аанар, Мсквск блсть внгрд, Моско..."
1,Ямало-Ненецкий АО Авангард,"[ЯмлоНеецийАОАвнгрд, мл-Ннцкй внгрд, ЯмалоНене..."
2,Республика Татарстан Авиатор,"[Репулиа атрсанАватр, Рспблк Ттрстн втр, Рспуб..."
3,Санкт-Петербург Аврора,"[СактПеерур Ароа, Снкт-Птрбрг врр, анкт-Петерб..."
4,Санкт-Петербург Ice Dream / Айс Дрим,"[СактПеерур Ie rem Ас ри, Снкт-Птрбрг Ice Dre..."
...,...,...
301,Алтайский край Прогресс,"[Алайки кайПргрсс, лтйскй крй Пргрсс, Алтаский..."
302,"Удмуртская республика ""СШ ""Гвоздика""","[Удурскя есубик ""Ш Гвздка, дмртск рспблк ""СШ ""..."
303,"Саратовская область СШОР ""Надежда Губернии","[Саатвсаяобась ШО ""аджд Гбени, Сртвск блсть СШ..."
304,Пермский край КФК «Айсберг»,"[Пемсийкрй ФК«Асбрг, Прмскй крй КФК «йсбрг», е..."


In [23]:
# Создадим эмбеддинги для эталонных написаний названий
corpus = model.encode(df['title'].values)

In [24]:
# Создадим эмбеддинги для аугментированных написаний названий
query = model.encode(test['augmented'].values)

In [25]:
search_result = util.semantic_search(query, corpus, top_k = 1)

In [26]:
# Извлечём id школ из корпуса
test['candidate_idx'] = [x[0]['corpus_id'] for x in search_result]

In [27]:
# Проверим результат
test

,title,augmented,candidate_idx
677,Москва ПроСинхро,Моска ПоСинхро,135
1497,"Москва ООО ""СетПоинт""","Моква ОО ""СетПоит""",299
927,Мурманская область СШ № 6,,279
131,Санкт-Петербург Буревестник,Снкт-Птрбрг Брвстнк,26
298,Иркутская область Ермак,Иркутсакя боласть Ерамк,59
...,...,...,...
768,Воронежская область Сияние,Вроонежская обалсть иСяние,153
250,Белгородская область ДЮСШ по ЗВС,Бегоодка олатьДЮШ о ВС,50
900,Вологодская область СШ № 1,Воогдсаяобась Ш 1,180
1106,Санкт-Петербург Темпо,Снкт-Птрбрг Тмп,221


In [28]:
# Добавим названий школ по id
test['candidate_name'] = df['title'].values[test['candidate_idx'].values]

In [29]:
# Смотрим на результат
test

,title,augmented,candidate_idx,candidate_name
677,Москва ПроСинхро,Моска ПоСинхро,135,Москва ПроСинхро
1497,"Москва ООО ""СетПоинт""","Моква ОО ""СетПоит""",299,"Москва ООО ""СетПоинт"""
927,Мурманская область СШ № 6,,279,Санкт-Петербург СШ (ОРК) филиала ФАУ МО РФ ЦСК...
131,Санкт-Петербург Буревестник,Снкт-Птрбрг Брвстнк,26,Санкт-Петербург Буревестник
298,Иркутская область Ермак,Иркутсакя боласть Ерамк,59,Иркутская область Ермак
...,...,...,...,...
768,Воронежская область Сияние,Вроонежская обалсть иСяние,153,Воронежская область Сияние
250,Белгородская область ДЮСШ по ЗВС,Бегоодка олатьДЮШ о ВС,50,Белгородская область ДЮСШ по ЗВС
900,Вологодская область СШ № 1,Воогдсаяобась Ш 1,180,Вологодская область СШ № 1
1106,Санкт-Петербург Темпо,Снкт-Птрбрг Тмп,221,Санкт-Петербург Темпо


Проверим соответствие, вычислим метрику Accuracy:

In [37]:
print('Accuracy:', ((test['title'] == test['candidate_name']).sum()/test.shape[0]).round(2))

Accuracy: 0.63


## Вывод

Результат можно улучшить с помощью добавления новых аугментированных названий